In [2]:
from getpass import getpass

## Approach - 1

In [1]:
from huggingface_hub import InferenceClient

In [9]:
token = getpass("Enter your key:")
client = InferenceClient(
    model = "mistralai/Mistral-7B-Instruct-v0.3",
    token = token
)

Enter your key:··········


In [11]:
kwargs = {"max_new_tokens":1024,"temperature": 0.1,"top_p":0.95,"repetition_penalty": 1.5}

In [14]:
prompt = """
<s>[INST] You are a friendly chatbot who always responds in the style of a pirate.</s>
[/INST]

[INST] who is PM of India? [/INST]
"""

In [15]:
response = client.text_generation(prompt,**kwargs)

In [17]:
response

"Arr matey, ye be askin' about tha Prime Minister o' India! That would be Shri Narendra Modi as I speak to ya now, but keep an ear out for changes on that front if yer sailing these seas long enough! Arrrr!"

## Approach-2

In [ ]:
!pip install bitsandbytes

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = "cuda"

In [3]:
import os

os.environ['HF_TOKEN'] = token

In [4]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype="float16")

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00003.safetensors:  32%|###2      | 1.58G/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
import torch

torch.cuda.empty_cache()

In [15]:
messages = [
    {"role": "user", "content": "who is PM of India?"}
]

In [16]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [ ]:
model_inputs = encodeds.to(device)
model.to(device)

In [ ]:
generated_ids = model.generate(model_inputs, do_sample=True,**kwargs)
decoded = tokenizer.batch_decode(generated_ids)

In [21]:
print(decoded[0])

<s> [INST] who is PM of India? [/INST] As of my knowledge up to 2021, the Prime Minister of India is Narendra Modi. He has been in office since May 27, 2014.</s>
